# Trabalho Final - Análise de Dados com Python

Integrantes:

1) Alison Junior Ghedin - 345377
2) Leonardo Sarzedas do Carmo Vieira - 345946

### Enunciado

A partir dos dados housing.csv, realize toda análise exploratória necessária e prepare os dados para algoritmos de machine learning usando a Scikit-Learn. Treine um modelo de regressão linear que prediga a variável meta (median_house_value) e cujo valor de RMSE seja menor ou igual a 69000. 

In [256]:
# leitura dos dados
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
housing = pd.read_csv('bases/housing.csv')
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [257]:
# linhas e colunas
housing.shape

(20640, 10)

In [258]:
# tipos de dados
housing.dtypes

longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

In [259]:
# estatísticas
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [260]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [261]:
# nulls
housing.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [262]:
# Add total_bedrooms when is null
housing['total_bedrooms'].fillna(housing['total_bedrooms'].mean(), inplace=True)
housing.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [263]:
# count value counts for each column
housing['ocean_proximity'].value_counts()


<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [264]:
# normalize ocean_proximity
housing['ocean_proximity'] = housing['ocean_proximity'].map({
    '<1H OCEAN': 1,
    'INLAND': 2,
    'NEAR OCEAN': 3,
    'NEAR BAY': 4,
    'ISLAND': 5
})
housing['ocean_proximity'].value_counts()

1    9136
2    6551
3    2658
4    2290
5       5
Name: ocean_proximity, dtype: int64

In [265]:
# Como só tem 5 valores em ISLAND, vamos remove-los
housing = housing.drop(housing[housing['ocean_proximity'] == 5].index)
housing['ocean_proximity'].value_counts()

1    9136
2    6551
3    2658
4    2290
Name: ocean_proximity, dtype: int64

In [266]:
# Media de pessoas por familia na região
housing['median_population_per_household'] = round(housing['population'] / housing['households'], 2)
# # Média de comodos por Familia
housing['median_rooms_per_household'] = round(housing['total_rooms'] / housing['households'], 2)
# # Média de quartos por Familia
housing['median_population_per_troms'] = round(housing['population'] / housing['total_rooms'], 2)
# # Proporção de Populalção por familia
housing['median_population_per_tbedrooms'] = round(housing['population'] / housing['total_bedrooms'], 2)
# # Média de aluguel por idade da casa
housing['median_income_per_houseage'] = round(housing['median_income'] / housing['housing_median_age'], 2)
# # Média de aluguel por total de quartos
housing['median_income_per_tbrooms'] = round(housing['median_income'] / housing['total_bedrooms'], 2)
# # Média de aluguel por familia
housing['median_income_per_households'] = round(housing['median_income'] / housing['households'], 2)
# # Proporção de Comodos por Idade da casa
housing['median_trooms_per_houseage'] = round(housing['total_rooms'] / housing['housing_median_age'], 2)
# Proporção de Quartos por Idade da casa
housing['median_tbrooms_per_houseage'] = round(housing['total_bedrooms'] / housing['housing_median_age'], 2)

In [267]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,median_population_per_household,median_rooms_per_household,median_population_per_troms,median_population_per_tbedrooms,median_income_per_houseage,median_income_per_tbrooms,median_income_per_households,median_trooms_per_houseage,median_tbrooms_per_houseage
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,4,2.56,6.98,0.37,2.50,0.20,0.06,0.07,21.46,3.15
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,4,2.11,6.24,0.34,2.17,0.40,0.01,0.01,338.05,52.67
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,4,2.80,8.29,0.34,2.61,0.14,0.04,0.04,28.21,3.65
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,4,2.55,5.82,0.44,2.37,0.11,0.02,0.03,24.50,4.52
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,4,2.18,6.28,0.35,2.02,0.07,0.01,0.01,31.29,5.38


In [268]:
# Scikit-Learn. Treine um modelo de regressão linear que prediga a variável meta (median_house_value) e cujo valor de RMSE seja menor ou igual a 69000. 
import numpy as np
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
X_train = housing.drop(['median_house_value'], axis=1)
y_train = housing['median_house_value']
lr.fit(X_train, y_train)
round(lr.score(X_train, y_train) * 100, 2)


64.36

In [ ]:
# RSME
from sklearn.metrics import mean_squared_error
housing_predictions = lr.predict(X_train)
housing_mse = mean_squared_error(y_train, housing_predictions)
housing_rmse = np.sqrt(housing_mse)
round(housing_rmse, 2)